In [1]:
'''
Transfer Learning - Problema cats x dogs
Implementação de rede convolucional usando transfer learning para diferenciação das categorias gato e cachorro.

O banco de dados original se encontra aqui. Dentre essas imagens foram separadas 8000 imagens para treinamento e 2000 imagens para teste.

Todo o código foi feito no Colab e pode ser compilado online em https://colab.research.google.com.

** Autor: Maicon Henrique Cunha**

Github: https://github.com/cunhamaicon

!rm -rf catsxdogs

!git clone https://github.com/cunhamaicon/catsxdogs
'''



'\nTransfer Learning - Problema cats x dogs\nImplementação de rede convolucional usando transfer learning para diferenciação das categorias gato e cachorro.\n\nO banco de dados original se encontra aqui. Dentre essas imagens foram separadas 8000 imagens para treinamento e 2000 imagens para teste.\n\nTodo o código foi feito no Colab e pode ser compilado online em https://colab.research.google.com.\n\n** Autor: Maicon Henrique Cunha**\n\nGithub: https://github.com/cunhamaicon\n\n!rm -rf catsxdogs\n\n!git clone https://github.com/cunhamaicon/catsxdogs\n'

In [2]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dropout
import scipy

In [3]:
'''
CNN - Transfer learning:
Importando o modelo MobileNet que foi previamente treinado no ImageNet e descartando a última camada de neurônios:'''

model=MobileNet(weights='imagenet',include_top=False)

In [4]:
#Criando a saída do modelo MobileNet:
x=model.output
x=GlobalAveragePooling2D()(x)
#Adicionando uma camada intermediária e a camada final:
x=Dense(50,activation='relu')(x)
preds=Dense(1,activation='sigmoid')(x) 
model=Model(inputs=model.input,outputs=preds)
#Visualizando todas as camadas da nova rede criada usando o modelo MobileNetV2:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_1
1 conv1
2 conv1_bn
3 conv1_relu
4 conv_dw_1
5 conv_dw_1_bn
6 conv_dw_1_relu
7 conv_pw_1
8 conv_pw_1_bn
9 conv_pw_1_relu
10 conv_pad_2
11 conv_dw_2
12 conv_dw_2_bn
13 conv_dw_2_relu
14 conv_pw_2
15 conv_pw_2_bn
16 conv_pw_2_relu
17 conv_dw_3
18 conv_dw_3_bn
19 conv_dw_3_relu
20 conv_pw_3
21 conv_pw_3_bn
22 conv_pw_3_relu
23 conv_pad_4
24 conv_dw_4
25 conv_dw_4_bn
26 conv_dw_4_relu
27 conv_pw_4
28 conv_pw_4_bn
29 conv_pw_4_relu
30 conv_dw_5
31 conv_dw_5_bn
32 conv_dw_5_relu
33 conv_pw_5
34 conv_pw_5_bn
35 conv_pw_5_relu
36 conv_pad_6
37 conv_dw_6
38 conv_dw_6_bn
39 conv_dw_6_relu
40 conv_pw_6
41 conv_pw_6_bn
42 conv_pw_6_relu
43 conv_dw_7
44 conv_dw_7_bn
45 conv_dw_7_relu
46 conv_pw_7
47 conv_pw_7_bn
48 conv_pw_7_relu
49 conv_dw_8
50 conv_dw_8_bn
51 conv_dw_8_relu
52 conv_pw_8
53 conv_pw_8_bn
54 conv_pw_8_relu
55 conv_dw_9
56 conv_dw_9_bn
57 conv_dw_9_relu
58 conv_pw_9
59 conv_pw_9_bn
60 conv_pw_9_relu
61 conv_dw_10
62 conv_dw_10_bn
63 conv_dw_10_relu
64 conv_pw_10
65 conv_pw_1

In [5]:
#Definindo qual camada da rede será treinada. Nesse caso somente as duas últimas camadas adicionadas:

for layer in model.layers[:88]:
    layer.trainable=False
for layer in model.layers[88:]:
    layer.trainable=True

###ImageDataGenerator

#Definindo o tamanho de cada batch:

batch_size = 32

#Cada imagem do banco será apresentada a rede de uma forma diferente através do ImageDataGenerator:

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.4,
                                   zoom_range = 0.4,
                                   height_shift_range=0.3,
                                   width_shift_range=0.3,
                                   rotation_range=50,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('catsxdogs/training_set',
                                                 target_size = (224, 224),
                                                 batch_size = batch_size,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('catsxdogs/test_set',
                                            target_size = (224, 224),
                                            batch_size = batch_size,
                                            class_mode = 'binary')


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [6]:
###Treinamento

#Definindo os parâmetros de compilação da rede:

model.compile(optimizer=Adam(learning_rate = 0.0001),loss='binary_crossentropy',metrics=['accuracy'])

#Fazendo o treinamento da rede:

history = model.fit(x=training_set,
                   steps_per_epoch=8000/batch_size,
                   epochs=10,
                   validation_data = test_set,
                   validation_steps = 2000/batch_size)

Epoch 1/10
250/250 [==============================] - 154s 611ms/step - loss: 0.8827 - accuracy: 0.3741 - val_loss: 0.9664 - val_accuracy: 0.3060
Epoch 2/10
250/250 [==============================] - 122s 488ms/step - loss: 0.8571 - accuracy: 0.3932 - val_loss: 0.9215 - val_accuracy: 0.3370
Epoch 3/10
250/250 [==============================] - 122s 489ms/step - loss: 0.8230 - accuracy: 0.4226 - val_loss: 0.8804 - val_accuracy: 0.3715
Epoch 4/10
250/250 [==============================] - 122s 489ms/step - loss: 0.7955 - accuracy: 0.4509 - val_loss: 0.8407 - val_accuracy: 0.4065
Epoch 5/10
250/250 [==============================] - 124s 497ms/step - loss: 0.7675 - accuracy: 0.4728 - val_loss: 0.8043 - val_accuracy: 0.4385
Epoch 6/10
250/250 [==============================] - 126s 504ms/step - loss: 0.7489 - accuracy: 0.4964 - val_loss: 0.7690 - val_accuracy: 0.4725
Epoch 7/10
250/250 [==============================] - 124s 497ms/step - loss: 0.7225 - accuracy: 0.5291 - val_loss: 0.7369 -

In [8]:
# Salvando o modelo para utilização futura:

model.save('catsxdogs_mobilenet.h5')
from IPython.display import FileLink

# Gerar um link de download para o arquivo
FileLink('catsxdogs_mobilenet.h5')

C:\Users\orlan\OneDrive\Documentos\UFF\DIO\ML\catsxdogs_mobilenet.h5

In [10]:
#Previsão

#Mostrando os arquivos da pasta single_prediction com imagens inéditas para a rede classificar:

import os

# Listar os arquivos na pasta single_prediction
files = os.listdir('catsxdogs/single_prediction')
print(files)


['cat_or_dog_1.jpg', 'cat_or_dog_2.jpg', 'chino1.jpg', 'floyd1.jpg', 'floyd2.jpg', 'floyd3.jpg', 'floyd4.jpg']


In [22]:
#Escolhendo uma imagem da pasta single_prediction para fazer a previsão:

test_image = image.load_img('catsxdogs/single_prediction/floyd1.jpg', target_size = (224, 224))

test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
test_image = test_image/255

result = model.predict(test_image)

if result[0][0] > 0.5:
    prediction = 'dog'
else:
    prediction = 'cat'
    
print(prediction)

In [24]:
#A accúracia do modelo ainda está muito baixa, vou treinar por mais 10 epochs

# Carregar o modelo salvo (caso você tenha fechado o notebook e deseje carregar novamente o modelo)
from keras.models import load_model

# Carregar o modelo treinado
model = load_model('catsxdogs_mobilenet.h5')

# Definir o número total de epochs desejado (10 anteriores + 10 adicionais)
total_epochs = 20

# Continuar o treinamento do modelo
history = model.fit(x=training_set,
                    steps_per_epoch=8000/batch_size,
                    epochs=total_epochs,
                    validation_data=test_set,
                    validation_steps=2000/batch_size)

Epoch 1/20
250/250 [==============================] - 122s 486ms/step - loss: 0.6493 - accuracy: 0.6205 - val_loss: 0.6234 - val_accuracy: 0.6550
Epoch 2/20
250/250 [==============================] - 124s 497ms/step - loss: 0.6296 - accuracy: 0.6415 - val_loss: 0.5996 - val_accuracy: 0.6835
Epoch 3/20
250/250 [==============================] - 120s 481ms/step - loss: 0.6161 - accuracy: 0.6589 - val_loss: 0.5767 - val_accuracy: 0.7120
Epoch 4/20
250/250 [==============================] - 118s 471ms/step - loss: 0.6018 - accuracy: 0.6883 - val_loss: 0.5553 - val_accuracy: 0.7390
Epoch 5/20
250/250 [==============================] - 118s 473ms/step - loss: 0.5873 - accuracy: 0.6966 - val_loss: 0.5356 - val_accuracy: 0.7645
Epoch 6/20
250/250 [==============================] - 118s 474ms/step - loss: 0.5780 - accuracy: 0.7039 - val_loss: 0.5174 - val_accuracy: 0.7830
Epoch 7/20
250/250 [==============================] - 117s 470ms/step - loss: 0.5620 - accuracy: 0.7266 - val_loss: 0.4994 -

In [31]:
# Salvando o modelo para utilização futura:

model.save('catsxdogsv30_mobilenet.keras')



In [29]:

#Refazendo os testes

#Escolhendo uma imagem da pasta single_prediction para fazer a previsão:

test_image = image.load_img('catsxdogs/single_prediction/cat_or_dog_1.jpg', target_size = (224, 224))

test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
test_image = test_image/255

result = model.predict(test_image)

if result[0][0] > 0.5:
    prediction = 'dog'
else:
    prediction = 'cat'
    
print(prediction)

1/1 [==============================] - 0s 25ms/step
cat
